In [1]:
# BRANCH="main"
BRANCH = 'r1.10.0'

In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.10.0) to /tmp/pip-install-6pg2jz9b/nemo-toolkit_55d603fe568a4d3d9ef80b541d7322f3
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-6pg2jz9b/nemo-toolkit_55d603fe568a4d3d9ef80b541d7322f3
  Running command git checkout -b r1.10.0 --track origin/r1.10.0
  Switched to a new branch 'r1.10.0'
  Branch 'r1.10.0' set up to track remote branch 'r1.10.0' from 'origin'.
  Resolved https://github.com/NVIDIA/NeMo.git to commit c3c843fd09208752ab527b79002b513a7e195753
  Created wheel for nemo-toolkit: filename=nemo_toolkit-1.10.0-py3-none-any.whl size=3328839 sha256=d814fd6035cc132b1606cf377857e1f2787f839f89361f1a01dfe44339064c36
  Stored in directory: /tmp/pip-ephem-wheel-cache-z5jeqbkf/wheels/19/ea/11/9b22a65269a5abb6bdbcdc76a7cdc76480d13a2a8176e3ffba
Successfully built nemo-toolkit
  Attempting uninstall: nemo-toolkit
    Found exis

# Introduction

In this notebook we demonstrate how to use p-tuning and prompt tuning within NeMo-Megatron. Both methods are parameter efficient alternatives to fine-tuning pretrained language models. Our NeMo implementation makes it possible to use one pretrained GPT model on many downstream tasks without needing to tune the model’s full set of parameters. It also allows for adding new tasks to your model without overwriting or disrupting previous tasks for which the model has already been p-tuned/prompt-tuned. Because the original model parameters are frozen and never altered by either method, p-tuning/prompt-tuning also avoid cartographic forgetting issues often encountered when fine-tuning models.

- Our prompt tuning implementation is based off Lester et. al’s EMNLP 2021 paper [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/abs/2104.08691)

- Our p-tuning implementation is based off Liu et al's paper [GPT Understands, Too](https://arxiv.org/abs/2103.10385).

- Usage examples and API documentation can be found in [our user docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/prompt_learning.html). 

<img src="images/prompt_learning_forward_pass.png" alt="Prompt Learning Forward Pass"/>

Our continuous learning capability for combined p-tuning and prompt tuning with GPT style models is a NeMo specific extension of the author’s original work.

# The Plan

We are going to show you how to:
    
    1. P-Tune/Prompt Tune a model on multiple tasks at the same time
    2. Add a new task to a model that has already been P-Tuned/Prompt Tuned previously
    
We will first p-tune a GPT model on sentiment analysis, and intent and slot classification tasks. Then we will show how to add the squad question answering task to the same model we already p-tuned once.


# Technical Overview
Instead of selecting discrete text prompts in a manual or automated fashion, prompt tuning and p-tuning utilize virtual prompt embeddings that can be optimized via gradient decent. The only difference between prompt tuning and p-tuning within NeMo-Megatron is the architecture used to tune the soft prompt tokens during training.

### Terminology
We will be using the terms `continuous`, `soft`, and `virtual` token interchangeably to refer to embeddings inserted into the model prompt that have no concrete mapping to strings or characters within the model’s vocabulary. These virtual token embeddings exist in contrast to the `discrete`, `hard`, or `real` tokens that do make up the model’s vocabulary. Virtual tokens are purely 1D vectors with dimensionality equal to that of each real token embedding, matching the `hidden_size` hyperparameter. In training and inference, continuous token embeddings are inserted among discrete token embeddings according to a template you provide in the model’s config. We will demonstrate how to do this below.

When referring to p-tuning and prompt tuning together, we will be using the phrase prompt learning for simplicity.

### Prompt-Tuning
In prompt-tuning a pretrained GPT model, soft prompt embeddings are initialized as a 2D matrix of size `total_virtual_tokens X hidden_size`. Each task the model is prompt-tuned to perform has its own 2D embedding matrix associated with it. Tasks do not share any parameters during training or inference. All GPT model parameters are frozen and only the embedding parameters for each task are updated during training.

In prompt tuning you can specify how the embeddings are initialized for each task. You can either

1. Initialize embedding parameters according to some random distribution
2. Initialize embedding parameters from existing vocabulary embeddings (recommended)

If you choose to initialize virtual token embeddings from existing embedding weights, you can provide the string of words you want to use for initialization in the model’s config. This string will be tokenized and tiled or truncated to match the specified number of virtual tokens you would like to use (`total_virtual_tokens`). Vocab embeddings are copied and used to initialize the soft prompt embedding matrix for each task. The vocab embeddings themselves are not updated or changed during prompt tuning.


### P-Tuning
In p-tuning, an LSTM model is used to predict virtual token embeddings. We refer to this LSTM model as our `prompt_encoder`. LSTM parameters are randomly initialized at the start of p-tuning. All GPT model parameters are frozen, and only the LSTM weights are updated at each training step. LSTM parameters are shared between all tasks that are p-tuned at the same time, but the LSTM model outputs unique virtual token embeddings for each task. The virtual tokens predicted by the LSTM are inserted among the discrete token input in the exact same manner as with prompt-tuning. You still specify the number of virtual tokens you want to use by setting `total_virtual_tokens` and each virtual token embedding is still a 1D vector of size `hidden_size`.



# The Best of Both
A single pretrained GPT model can use both p-tuning and prompt-tuning. While you must decide to use either p-tuning or prompt-tuning for each task you want your model to perform, you can p-tune your model on a set of tasks A, then prompt tune your same model on a different set of tasks B, then finally run inference on tasks from both A and B at the same time. During prompt-tuning or p-tuning, tasks tuned at the same time must use the same number of virtual tokens. During inference, tasks using differing amounts of virtual tokens can be run at the same time.

Please see our [docs for more comparisons between prompt and p-tuning](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/prompt_learning.html). 

With all that covered, let's get started!


In [3]:
import os
import wget 
import pathlib

# Tasks and Datasets
We will be using p-tuning to teach our GPT model to do 3 tasks: **Sentiment Analysis**, **Question Answering** and **Intent and Slot Classification**.

We will use [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for our sentiment analysis task, [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) for question answering task, and the [Assistant Benchmarking Dataset](https://github.com/xliuhw/NLU-Evaluation-Data) for intent and slot classification. 

- The [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) contains the sentiments for financial news headlines from the perspective of a retail investor. Further details about the dataset can be found in Malo et. al's "[Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts](https://arxiv.org/abs/1307.5336)"


- [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) is a reading comprehension dataset, consisting of questions posed by crowd workers on a set of Wikipedia articles, where the answer to every question is a segment of text. More information on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) can be found on their website or in their paper by Rajpurkar et. al "[Know What You Don’t Know: Unanswerable Questions for SQuAD](https://arxiv.org/pdf/1806.03822.pdf)".


- The [Assistant Benchmarking Dataset](https://github.com/xliuhw/NLU-Evaluation-Data) is a natural language dataset for in home human-robot interaction. Details on the dataset can be found in Liu et. al's "[Benchmarking Natural Language Understanding Services for building Conversational Agents](https://arxiv.org/abs/1903.05566)"

Each of these tasks require different types of natural language understanding and lie in different domains. We will demonstrate how to use one model to perform all of them.

# Data Preparation

The prompt learning dataset loader accepts a list of json/dictionary objects or a list of json file names where each json file contains a collection of json objects. Each json object must include the field `taskname` which is a string identifier for the task the data example corresponds to. They should also include one or more fields corresponding to different sections of the discrete text prompt. The input data might look like:

```
[
    {"taskname": "squad", "context": [CONTEXT_PARAGRAPH_TEXT1], "question": [QUESTION_TEXT1], "answer": [ANSWER_TEXT1]},
    {"taskname": "squad", "context": [CONTEXT_PARAGRAPH_TEXT2], "question": [QUESTION_TEXT2], "answer": [ANSWER_TEXT2]},
    {"taskname": "intent_and_slot", "utterance": [UTTERANCE_TEXT1], "label": [INTENT_TEXT1][SLOT_TEXT1]},
    {"taskname": "intent_and_slot", "utterance": [UTTERANCE_TEXT2], "label": [INTENT_TEXT2][SLOT_TEXT2]},
    {"taskname": "sentiment", "sentence": [SENTENCE_TEXT1], "label": [SENTIMENT_LABEL1]},
    {"taskname": "sentiment", "sentence": [SENTENCE_TEXT2], "label": [SENTIMENT_LABEL2]},
]
```

These additional fields can be unlimited in number and will be used to help map different parts of the discrete text input to a prompt template that you define. We will show how this mapping works and how to construct your prompt template in the `Prompt Formatting` section. 

In [4]:
# You can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
NEMO_DIR = '.'

os.makedirs(DATA_DIR, exist_ok=True)


For each dataset we have preprocessing scripts pre-written in NeMo's example directory located in `examples/nlp`. Let's download those now. 

In [5]:
# download the preprocessing scripts from github for the purpose of this tutorial
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/nlp/financial_phrase_bank/prompt_learning_financial_phrase_bank_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/nlp/squad/prompt_learning_squad_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/nlp/intent_and_slot/prompt_learning_assistant_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/nlp/intent_and_slot/assistant_utils.py', NEMO_DIR)

'./assistant_utils.py'

Now let's down load and process each dataset.

###  Financial PhraseBank Dataset

In [6]:
# Download the financial phrase bank dataset
!wget https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v1.0/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v1.0.zip

# If you are having issues with the research gate link above, copy and paste it in your browser 
# and the file should download automatically. Then place it in the same directory in which 
# you are running this notebook. 

--2022-07-12 22:02:29--  https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v1.0/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v1.0.zip
Resolving www.researchgate.net (www.researchgate.net)... 104.17.32.105, 104.17.33.105, 2606:4700::6811:2069, ...
Connecting to www.researchgate.net (www.researchgate.net)|104.17.32.105|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2022-07-12 22:02:29 ERROR 403: Forbidden.



If you are having issues with the research gate link above, copy and paste it in your browser and the file should download automatically. Then place it in the same directory in which you are running this notebook. 

In [14]:
# !unzip FinancialPhraseBank-v1.0.zip -d {DATA_DIR}

In [15]:
# What the financial phrase bank dataset looks like before processing
SENTIMENT_DIR = os.path.join(DATA_DIR, "FinancialPhraseBank-v1.0")
!head -4 $SENTIMENT_DIR/Sentences_AllAgree.txt

According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral
For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .@positive
In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .@positive
Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .@positive


In [16]:
# Preprocess financial phrase bank dataset
!python $NEMO_DIR/prompt_learning_financial_phrase_bank_preprocessing.py

Saving train split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl
100%|███████████████████████████████████| 1811/1811 [00:00<00:00, 263489.82it/s]
Saving val split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl
100%|█████████████████████████████████████| 226/226 [00:00<00:00, 250505.47it/s]
Saving test split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_test.jsonl
100%|█████████████████████████████████████| 227/227 [00:00<00:00, 271332.86it/s]


In [17]:
# What the financial phrase bank dataset looks like after processing
!head -4 $SENTIMENT_DIR/financial_phrase_bank_train.jsonl

{"taskname": "sentiment", "sentence": "The new organization consists of two business units : Charging & Messaging and Finance & Administration .", "label": " neutral"}
{"taskname": "sentiment", "sentence": "Danske Bank is Denmark 's largest bank with 3.5 million customers .", "label": " neutral"}
{"taskname": "sentiment", "sentence": "The total value of the deliveries is some EUR65m .", "label": " neutral"}
{"taskname": "sentiment", "sentence": "In future , the plant will focus on the production of flange profiles for wind farm towers .", "label": " neutral"}


Our financial phrase bank preprocessing script converted the raw text file of sentences and labels into three `.jsonl` files for training, validation, and testing. Each line in the files contains a json object with the fields `taskname`, `sentiment`, `sentence`, and `label`. You can inspect the preprocessing script and play with different arguments for the script by looking at and running `prompt_learning_financial_phrase_bank_preprocessing.py` which should currently be downloaded in `NEMO_DIR`. It is also located at `scripts/dataset_processing/nlp/financial_phrase_bank/prompt_learning_financial_phrase_bank_preprocessing.py` in the NeMo repo.

By default 80% of the data was randomly selected for the training set, 10% for the validation set, and 10% for the test set. We only used training examples with 100% agreement from labelers on the correct sentiment label. This data is from `Sentences_AllAgree.txt`. This should result in `1811` training examples, `226` validation examples, and `227` examples for testing. The `label` field was removed from test examples. 

If you want to try using more data, you can combine the `Sentences_AllAgree.txt` with any of the `Sentences_75Agree.txt`, `Sentences_66Agree.txt` and/or `Sentences_50Agree.txt` by creating a new cell below and running:

```
!cat {SENTIMENT_DIR}/Sentences_AllAgree.txt {SENTIMENT_DIR}/Sentences_75Agree.txt >> {SENTIMENT_DIR}/combined_data.txt
!python $NEMO_DIR/prompt_learning_financial_phrase_bank_preprocessing.py --file-name combined_data.txt
```


### SQuAD Dataset

In [18]:
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")
os.makedirs(SQUAD_DIR, exist_ok=True)

# Download the SQuAD dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!mv train-v2.0.json {SQUAD_DIR}

--2022-07-12 22:16:57--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   108MB/s    in 0.4s    

2022-07-12 22:16:59 (108 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [19]:
# Preprocess squad data
!python $NEMO_DIR/prompt_learning_squad_preprocessing.py --data-dir {SQUAD_DIR}

Saving train split to data/SQuAD/squad_train.jsonl
100%|█████████████████████████████████| 69125/69125 [00:00<00:00, 145756.95it/s]
Saving val split to data/SQuAD/squad_val.jsonl
100%|███████████████████████████████████| 8952/8952 [00:00<00:00, 160849.45it/s]
Saving test split to data/SQuAD/squad_test.jsonl
100%|███████████████████████████████████| 8744/8744 [00:00<00:00, 164360.88it/s]


In [20]:
# What the squad dataset looks like after processing
!head -4 $SQUAD_DIR/squad_train.jsonl

{"taskname": "squad", "context": "The national bird of Saint Helena is the Saint Helena plover, known locally as the wirebird. It appears on the coat of arms of Saint Helena and on the flag.", "question": "What is the national bird of Saint Helena?", "answer": " the Saint Helena plover"}
{"taskname": "squad", "context": "Other than by appearance, \"Coloureds\" can usually be distinguished from \"Blacks\" by language. Most speak Afrikaans or English as a first language, as opposed to Bantu languages such as Zulu or Xhosa. They also tend to have more European-sounding names than Bantu names.", "question": "What type of names do \"blacks\" have?", "answer": " Bantu names"}
{"taskname": "squad", "context": "The history of the Ottoman Empire during World War I began with the Ottoman engagement in the Middle Eastern theatre. There were several important Ottoman victories in the early years of the war, such as the Battle of Gallipoli and the Siege of Kut. The Arab Revolt which began in 1916 t

We made a `.jsonl` file for each of the train, validation, and testing splits of the squad data. Every `.jsonl` file contains json objects with the fields `taskname`, `context`, `question`, and `answer`. The preprocessing script is called `prompt_learning_squad_preprocessing.py`. It should be in your `NEMO_DIR` and at `scripts/dataset_processing/nlp/squad/prompt_learning_squad_preprocessing.py` in the NeMo repo. 

The SQuAD dataset consists of various topics like `Beyoncé`, `IPod`, and `Symbiosis`. Each topic has several paragraphs associated with it, and each paragraph has several questions and answers related to it. When we separated the train/validation/test splits, we separated them on the topic level. For example, if the training set contains paragraphs and questions about the topic `Beyoncé`, neither the validation nor test sets will contain any questions on this topic. All questions about a certain topic are isolated to one split of the data. 

Like the Financial PhraseBank Dataset, we randomly selected 80% of the questions for training, 10% for validation, and 10% for test. This resulted in `69125` test examples, `8952` validation examples, and `8744` testing examples. The `answer` field was removed from test examples.

Training on the full train split could take a lot of time, so we are going to clip the train split to 20k examples for the sake of this tutorial. 

In [21]:
! head -20000 $SQUAD_DIR/squad_train.jsonl > $SQUAD_DIR/squad_short_train.jsonl

### Assistant Dataset

In [22]:
ASSISTANT_DIR = os.path.join(DATA_DIR, "assistant")
os.makedirs(ASSISTANT_DIR, exist_ok=True)

# Download the assisent dataset
!wget https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip
!unzip master.zip -d {ASSISTANT_DIR}

--2022-07-12 22:17:13--  https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/xliuhw/NLU-Evaluation-Data/zip/refs/heads/master [following]
--2022-07-12 22:17:13--  https://codeload.github.com/xliuhw/NLU-Evaluation-Data/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [             <=>    ]  20.53M  8.01MB/s    in 2.6s    

2022-07-12 22:17:16 (8.01 MB/s) - ‘master.zip’ saved [21527506]

Archive:  master.zip
f6071b496b17d71e6eb43f543af0707f4ff30557
   creating: data/assistant/NLU-Evaluation-Data-master/
 

In [23]:
# Process virtual assistant intent and slot classification data
!python $NEMO_DIR/prompt_learning_assistant_preprocessing.py

[NeMo W 2022-07-12 22:17:24 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
      warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
    
[NeMo W 2022-07-12 22:17:27 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-12 22:17:27 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-07-12 22:17:27 assistant_utils:141] Processing assistant commands dataset and store at data/assistant/nemo-format
[NeMo I 2022-07-12 22:17:27 assistant_utils:36] Copying files to input folder: data/assist

In [24]:
!head -5 $ASSISTANT_DIR/assistant_train.jsonl

{"taskname": "intent_and_slot", "utterance": "who was john dillinger", "label": "\nIntent: qa_factoid\nSlots: person(john dillinger)"}
{"taskname": "intent_and_slot", "utterance": "can you play my favorite music", "label": "\nIntent: play_music\nSlots: None"}
{"taskname": "intent_and_slot", "utterance": "is adele going to go on tour", "label": "\nIntent: qa_factoid\nSlots: artist_name(adele)"}
{"taskname": "intent_and_slot", "utterance": "will the temperature be in the today", "label": "\nIntent: weather_query\nSlots: weather_descriptor(temperature), date(today)"}
{"taskname": "intent_and_slot", "utterance": "clap on", "label": "\nIntent: iot_hue_lighton\nSlots: None"}


For the virtual assistant dataset, there are a set of 64 possible intents:

In [25]:
!echo 'Intents: ' $(wc -l < {ASSISTANT_DIR}/nemo-format/dict.intents.csv)
!cat {ASSISTANT_DIR}/nemo-format/dict.intents.csv

Intents:  64
alarm_query
alarm_remove
alarm_set
audio_volume_down
audio_volume_mute
audio_volume_up
calendar_query
calendar_remove
calendar_set
cooking_recipe
datetime_convert
datetime_query
email_addcontact
email_query
email_querycontact
email_sendemail
general_affirm
general_commandstop
general_confirm
general_dontcare
general_explain
general_joke
general_negate
general_praise
general_quirky
general_repeat
iot_cleaning
iot_coffee
iot_hue_lightchange
iot_hue_lightdim
iot_hue_lightoff
iot_hue_lighton
iot_hue_lightup
iot_wemo_off
iot_wemo_on
lists_createoradd
lists_query
lists_remove
music_likeness
music_query
music_settings
news_query
play_audiobook
play_game
play_music
play_podcasts
play_radio
qa_currency
qa_definition
qa_factoid
qa_maths
qa_stock
recommendation_events
recommendation_locations
recommendation_movies
social_post
social_query
takeaway_order
takeaway_query
transport_query
transport_taxi
transport_ticket
transport_traffic
weather_query


and 55 types of slots:

In [26]:
# print all slots from the NeMo format slot dictionary
!echo 'Slots: ' $(wc -l < {ASSISTANT_DIR}/nemo-format/dict.slots.csv)
!cat {ASSISTANT_DIR}/nemo-format/dict.slots.csv

Slots:  55
alarm_type
app_name
artist_name
audiobook_author
audiobook_name
business_name
business_type
change_amount
coffee_type
color_type
cooking_type
currency_name
date
definition_word
device_type
drink_type
email_address
email_folder
event_name
food_type
game_name
game_type
general_frequency
house_place
ingredient
joke_type
list_name
meal_type
media_type
movie_name
movie_type
music_album
music_descriptor
music_genre
news_topic
order_type
person
personal_info
place_name
player_setting
playlist_name
podcast_descriptor
podcast_name
radio_name
relation
song_name
time
time_zone
timeofday
transport_agency
transport_descriptor
transport_name
transport_type
weather_descriptor
O


Each slot label consists of the slot type followed by specific text from the utterance corresponding to that slot type in parentheses. For example, the utterance `"tell my facebook group that i've arrived"` has the intent label `social_post` and the slot label `media_type(facebook)`. Utterances each have one intent label and zero or more slot labels. In cases where there is no slot label, our GPT model should predict the word `None`. 

Json objects for each training example contain three fields: `taskname`, `utterance`, and `label`. For this dataset, our preprocessing script formatted our intent and slot labels to look like `"\nIntent: transport_taxi\nSlots: transport_agency(golden taxi), time(seven pm), date(today)"`. With newline characters (\n) separating intent and slot labels. Our train jsonl file has `9960` training examples. Our validation and test jsonl files each have `538` training examples. Test examples do not have the `label` field. 

The preprocessing script can be found at `scripts/dataset_processing/nlp/intent_and_slot/prompt_learning_assistant_preprocessing.py`

# P-Tuning Model Config Setup

Now we will begin setting up the config file used for prompt/p-tuning our GPT models! GPT Prompt learning within NeMo uses a class called `MegatronGPTPromptLearningModel` which has its own config file. We will start by loading an example prompt learning config file, then make changes to it to fit our tasks and training plans. 

In [27]:
from omegaconf import OmegaConf

CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
os.makedirs(CONFIG_DIR, exist_ok=True)

# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/megatron_gpt_prompt_learning_config.yaml', CONFIG_DIR)

# Load the example config file so we can start editing it
CONFIG_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_prompt_learning_config.yaml")
config = OmegaConf.load(CONFIG_PATH)

First let's set the datasets we've created in the config. We are going to start by p-tuning a GPT model on the **Sentiment Analysis** and **Intent and Slot Classification** tasks. We will set only those two datasets in the config file right now. 

In [28]:
config.model.data.train_ds = [f"{SENTIMENT_DIR}/financial_phrase_bank_train.jsonl", f"{ASSISTANT_DIR}/assistant_train.jsonl"]
config.model.data.validation_ds = [f"{SENTIMENT_DIR}/financial_phrase_bank_val.jsonl", f"{ASSISTANT_DIR}/assistant_val.jsonl"]

The `MegatronGPTPromptLearningModel` class expects datasets to be a list of `.json` or `.jsonl` file paths. You can give it multiple datasets at once like we did above.  

### Prompt Formatting
Now that we have our datasets set, lets define what we want the prompt for each task to look like. 

To customize different prompts for different tasks, we simply need to specify the prompt task template in the config file. The virtual token markers `<|VIRTUAL_PROMPT_#|>` signify where you want virtual tokens to be placed in the template string. `<|VIRTUAL_PROMPT_0|>`, `<|VIRTUAL_PROMPT_1|>`, and `<|VIRTUAL_PROMPT_2|>` indicate where a number of virtual tokens matching the values given at `virtual_token_splits[0]`, `virtual_token_splits[1]` and `virtual_token_splits[2]` will be placed. The other variable fields `{var}` refer to the fields in the data json. 

For example, given: 

1. the data json **{"sentence1": "And he said, Mama, I'm home.", "sentence2": "He didn't say a word."}**


2. virtual token splits set to `virtual_token_splits = [3, 3, 3]` 


3. a prompt template set to `prompt_template = "<|VIRTUAL_PROMPT_0|> Hypothesis: [sentence1], <|VIRTUAL_PROMPT_1|> Premise: [sentence2] <|VIRTUAL_PROMPT_2|> Answer:"`

the input will be translated into **<span style="color:red">VVV</span> Hypothesis: And he said, Mama, I'm home.<span style="color:red">VVV</span> Premise: He didn't say a word.<span style="color:red">VVV</span> Answer:**, where <span style="color:red">VVV</span> are three virtual tokens.

Because we are only p-tuning on the Sentiment Analysis and Intent/Slot Classification tasks right now, we only need to set the task templates for those two tasks. But, we are going to go ahead and set the template for all 3 tasks here, just to show that you can set the templates for all the tasks you have planned at one time, then prompt tune/p-tune on them sequentially. 

Let's configure all of our templates for **Sentiment Analysis**, **Intent and Slot Classification**, and **Question Answering** tasks:


In [29]:
config.model.task_templates = [
    {
        "taskname": "sentiment",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> {sentence} sentiment:{label}",
        "total_virtual_tokens": 10,
        "virtual_token_splits": [10],
        "truncate_field": None,
        "answer_only_loss": True,
        "answer_field": "label",
    },
    {
        "taskname": "intent_and_slot",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|> :\n{utterance}{label}",
        "total_virtual_tokens": 10,
        "virtual_token_splits": [7, 3],
        "truncate_field": None,
        "answer_only_loss": False,
    },
    {
        "taskname": "squad",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> Context: {context}\n\nQuestion: {question}\n\nAnswer:{answer}",
        "total_virtual_tokens": 15,
        "virtual_token_splits": [15],
        "truncate_field": "context",
        "answer_only_loss": True,
        "answer_field": "answer",
    },
    
]

Note each `task_template` item has 5 fields. 

- **`prompt_template`** is a string showing the model where to place virtual tokens and how to map dataset json fields to where they belong in the model prompt. 


- **`taskname`** refers to the same `taskname` in the dataset json objects. 


- **`total_virtual_tokens`** specifies the total number of virtual tokens that will be inserted into the model prompt.


- **`virtual_token_splits`** specifies the number of virtual tokens that belong at each `<|VIRTUAL_PROMPT_#|>` marker. `virtual_token_splits` values should add up to `total_virtual_tokens`. The number of `virtual_token_splits` should match the number of `<|VIRTUAL_PROMPT_#|>` markers. 


- **`truncate_field`** specifies which field in the data json to truncate if the length of the input exceeds the maximum sequence length of the model. If `truncate_field` is set to `None`, examples that are too long are simply dropped from the dataset.


- **`answer_only_loss`** Whether to limit loss calculation to only the answer portion of the prompt during tuning. `True` Strongly recommended for long prompts, but shorter prompts with single word answers seem to benefit from setting this to `False`. 


- **`answer_field`** The field in the data json corresponding to the answer. The loss will only be calculated on this portion of the prompt if `answer_only_loss` is `True`. The answer field must be at the end of the prompt template.

In the `task_templates` we set above, `squad` has a different number of virtual tokens than `sentiment` and `intent_and_slot`. This is because we will be p-tuning on `squad` after we p-tune on the other two tasks and **we do not need to use the same number of virtual tokens between sessions**. We also set the `truncate` field for squad because the context can sometimes be longer than the model's max sequence length, and we want that field to be truncated if the example is too long. Lastly, we set `answer_only_loss` to true for `squad` due to the longer prompt. We've found `answer_only_loss=True` to work significantly better for this task.

### Setting New Tasks
After you p-tune your model this time, you can always go back and p-tune or prompt-tune your model on more tasks without over writing the virtual prompts who've trained this time. You can also use a different number of `total_virtual_tokens` between each training session as long as tasks ptuned or prompt tuned at the same time have the same number of `total_virtual_tokens`. For this reason, when you p-tune on a new task, you need to tell your model which of your tasks are new and which ones already exist (and thus you don't want to tune them). 

You do this by setting the `new_tasks` and `existing_tasks` values in the config file. Because we are p-tuning a model with no existing tasks, you should set `existing_tasks=[]` and `new_tasks=["sentiment", "intent_and_slot"]` as follows:

In [30]:
config.model.existing_tasks = []
config.model.new_tasks = ["sentiment", "intent_and_slot"]

After p-tuning and/or prompt tuning is complete, you can run inference on all tasks at the same time, regardless of their `total_virtual_tokens` value.

### Setting The Pre-Trained GPT Model
We still need to set which GPT model we want to p-tune/prompt tune. Prompt learning methods work best with large GPT language models (5B or above), but the purposes of this tutorial, we are going to download a 345M parameter GPT model from NVIDIA NGC.

In [31]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
MegatronGPTModel.list_available_models()

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[NeMo W 2022-07-12 22:18:28 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
      warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
    
[NeMo W 2022-07-12 22:18:30 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-12 22:18:30 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully suppor

[PretrainedModelInfo(
 	pretrained_model_name=megatron_gpt_345m,
 	description=345M parameter GPT generative Megatron model.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
 )]

If we wanted to use the GPT model class directly, we could instantiate a trainer then download the model by calling running 
`gpt_model = MegatronGPTModel.from_pretrained(model_name="megatron_gpt_345m", trainer=trainer).cuda()`. But we just need the `.nemo` file in our working NeMo directory in this tutorial, so we will download it using `wget`. 

In [32]:
# Download the model from NGC
gpt_file_name = "megatron_gpt_345m.nemo"
!wget --content-disposition https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo -O {NEMO_DIR}/{gpt_file_name}

--2022-07-12 22:18:33--  https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 13.57.66.55, 52.9.115.177
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|13.57.66.55|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://prod-model-registry-ngc-bucket.s3.us-west-2.amazonaws.com/org/nvidia/team/nemo/models/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo?response-content-disposition=attachment%3B%20filename%3D%22megatron_gpt_345m.nemo%22&response-content-type=application%2Foctet-stream&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEC0aCXVzLXdlc3QtMSJHMEUCIDpY4kQdmv0cCFce9%2FXeQfVWu%2FlsJhROW8KRGJhbK%2FK6AiEA8Is%2FCRhwL%2BpJnimgIfgt%2Fbv6sV4VA7PbrIWPxh4P4z8q0gQIZhAEGgw3ODkzNjMxMzUwMjciDHOPtLoXrX2M2qSGRiqvBH1%2FPCN%2BEKsgoE%2FRxWmaTOkyNbiKphuRBXPRDv8T9I4hPu%2FfuYRD24xVaiMqY7lU7a1uRgvmV%2BESCeXvVVWHv2ZcxL2iM6Wz9SsZwMReB5aJ2cSRl3%2FK2s0sm3oVr%2FnMNa4V%2Bt

Now that we have a `.nemo` GPT file to work with. We need to add its path in our prompt learning config. 

In [33]:
# Set GPT model path on prompt learning config
config.model.language_model_path = gpt_file_name

We can also set where we want the final prompt tuned model to be saved by setting `model.nemo_path`. By default the tuned prompt learning model will be saved in your current working directory to a `.nemo` file with the same name as your experiment (`config.name`). Let's change the save name to be `multitask_p_tuned_gpt.nemo`. **Your model path must end in `.nemo`.**

In [34]:
config.model.nemo_path = "multitask_p_tuned_gpt.nemo"

### Setting P-Tuning Specific Params
Within the config file, p-tuning and prompt-tuning each have a couple of hyperparameters specific to them. We first need to tell the model that we want to do p-tuning, not prompt-tuning. To do this, we set the **`model.virtual_prompt_style`** hyperparameter like this:

In [35]:
from nemo.collections.nlp.modules.common import VirtualPromptStyle
config.model.virtual_prompt_style = VirtualPromptStyle.P_TUNING

Then we can set the 2 p-tuning specific parameters. Reminder, pp-tuning uses an LSTM prompt encoder to predict virtual tokens. 

- **`p_tuning.dropout`** the LSTM prompt encoder dropout probability 
- **`p_tuning.num_layers`** the number of LSTM layers you want your p-tuning prompt encoder to have


In [36]:
config.model.p_tuning.dropout = 0.0
config.model.p_tuning.num_layers = 2

Let's have a look at all the values we've set in the model config. You can change any of these values in the same manner we've been using above. 

In [37]:
# Final model config
print(OmegaConf.to_yaml(config.model))

seed: 1234
nemo_path: multitask_p_tuned_gpt.nemo
virtual_prompt_style: P_TUNING
encoder_seq_length: 2048
tensor_model_parallel_size: 1
pipeline_model_parallel_size: 1
global_batch_size: 8
micro_batch_size: 4
restore_path: null
language_model_path: megatron_gpt_345m.nemo
existing_tasks: []
new_tasks:
- sentiment
- intent_and_slot
task_templates:
- taskname: sentiment
  prompt_template: <|VIRTUAL_PROMPT_0|> {sentence} sentiment:{label}
  total_virtual_tokens: 10
  virtual_token_splits:
  - 10
  truncate_field: null
  answer_only_loss: true
  answer_field: label
- taskname: intent_and_slot
  prompt_template: '<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|>
    :

    {utterance}{label}'
  total_virtual_tokens: 10
  virtual_token_splits:
  - 7
  - 3
  truncate_field: null
  answer_only_loss: false
- taskname: squad
  prompt_template: '<|VIRTUAL_PROMPT_0|> Context: {context}


    Question: {question}


    Answer:{answer}'
  total_virtual_tokens: 15
  virtual_token_splits

### Setting Prompt-Tuning Specific Params

Though we are not using prompt tuning in this training session, lets go over the prompt tuning specific parameters we would use if we were. 

- **`prompt_tuning.new_prompt_init_methods`** Whether you want to initialize virtual token embeddings from the embeddings of existing parts of the model's vocabulary (either 'text' or 'random')
- **`prompt_tuning.new_prompt_init_text`** The text you want to use if you have 'text' in the list above, should be None otherwise. 

Each of the above hyperparameters are a list of strings. 

`new_prompt_init_methods` would look like `["text", "random", "text", "text"]` if you were prompt tuning on 4 tasks at once, and you wanted the second task in `new_tasks` to use random initialization. 

`new_prompt_init_text` might look like `["some text I want to use", None, "some other text", "task text goes here"]` for those four new tasks. 

The order of both should correspond to the order of the tasks you have listed in `model.new_tasks`. 

# Building the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [38]:
import torch
import pytorch_lightning as pl
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPPlugin
from pytorch_lightning.plugins.environments.torchelastic_environment import TorchElasticEnvironment

# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.accelerator = accelerator
config.trainer.devices = 1
config.trainer.max_epochs = 10
config.trainer.val_check_interval = 1.0

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
config.trainer.strategy = None

# setup cluster environment parameters"
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'

plugins = [NLPDDPPlugin(find_unused_parameters=False, no_ddp_communication_hook=True), TorchElasticEnvironment()]
trainer = pl.Trainer(plugins=plugins, **config.trainer)

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

[NeMo W 2022-07-12 22:21:02 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp.py:20: LightningDeprecationWarning: The `pl.plugins.training_type.ddp.DDPPlugin` is deprecated in v1.6 and will be removed in v1.8. Use `pl.strategies.ddp.DDPStrategy` instead.
      rank_zero_deprecation(
    
[NeMo W 2022-07-12 22:21:02 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:317: LightningDeprecationWarning: Passing <nemo.collections.nlp.parts.nlp_overrides.NLPDDPPlugin object at 0x7fe29cd4ccd0> `strategy` to the `plugins` flag in Trainer has been deprecated in v1.5 and will be removed in v1.7. Use `Trainer(strategy=<nemo.collections.nlp.parts.nlp_overrides.NLPDDPPlugin object at 0x7fe29cd4ccd0>)` instead.
      rank_zero_deprecation(
    
Using 16bit native Automatic Mixed Precision (AMP)
[NeMo W 2022-07-12 22:21:02 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytor

Trainer config - 

devices: 1
accelerator: gpu
num_nodes: 1
precision: 16
logger: false
enable_checkpointing: false
replace_sampler_ddp: false
max_epochs: 10
max_steps: null
log_every_n_steps: 10
val_check_interval: 1.0
accumulate_grad_batches: 1
gradient_clip_val: 1.0
resume_from_checkpoint: null
benchmark: false
strategy: null



# Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [39]:
from nemo.utils.exp_manager import exp_manager

# Set name of the experiment 
config.name = 'sentiment_intent_slot_p_tuning'
config.exp_manager.resume_if_exists = False

# Init the experiment manager and view the exp_dir
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)
print(exp_dir)

[NeMo I 2022-07-12 22:21:12 exp_manager:287] Experiments will be logged at /workspace/NeMo/tutorials/nlp/nemo_experiments/sentiment_intent_slot_p_tuning/2022-07-12_22-21-12
[NeMo I 2022-07-12 22:21:12 exp_manager:661] TensorboardLogger has been set up


[NeMo W 2022-07-12 22:21:12 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2319: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.")
    
[NeMo W 2022-07-12 22:21:12 exp_manager:895] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.
[NeMo W 2022-07-12 22:21:12 exp_manager:698] Found save_best_model is True and save_nemo_on_train_end is False. Set save_nemo_on_train_end to True to automatically save the best model.


/workspace/NeMo/tutorials/nlp/nemo_experiments/sentiment_intent_slot_p_tuning/2022-07-12_22-21-12


We can also set learning hyperparameters as follows:

In [40]:
# Set some of the learning parameters
config.model.optim.lr = 1e-4
config.model.precision = config.trainer.precision

# First P-Tuning Session
The only thing left to do is load up the model and begin p-tuning!

In [41]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_prompt_learning_model import MegatronGPTPromptLearningModel

model = MegatronGPTPromptLearningModel(cfg=config.model, trainer=trainer)

[NeMo I 2022-07-12 22:21:18 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-07-12 22:21:18 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-07-12 22:21:18 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-07-12 22:21:18 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-07-12 22:21:18 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-07-12 22:21:18 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-07-12 22:21:18 megatron_init:231] All tensor model parallel group ranks: [[0]]
[NeMo I 2022-07-12 22:21:18 megatron_init:232] Rank 0 has tensor model parallel rank: 0
[NeMo I 2022-07-12 22:21:18 megatron_init:246] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2022-07-12 22:21:18 megatron_init:258] Rank 0 has embedding group: [0]
[NeMo I 2022-07-12 22:21:18 megatron_init:264] All pipeline model parallel group ranks: [[0]]
[NeMo I 2022-07-12 22:21:18 megatron_init:265]

[NeMo W 2022-07-12 22:21:20 modelPT:215] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-07-12 22:21:20 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-gpt-345m, custom vocab file: /tmp/tmp3wt9a_ak/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, and merges file: /tmp/tmp3wt9a_ak/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt
[NeMo I 2022-07-12 22:21:20 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: gpt2, vocab_file: /tmp/tmp3wt9a_ak/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, merges_files: /tmp/tmp3wt9a_ak/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt, special_tokens_dict: {}, and use_fast: False


Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2022-07-12 22:21:23 megatron_base_model:161] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2022-07-12 22:21:24 nlp_overrides:367] Model MegatronGPTModel was successfully restored from /workspace/NeMo/tutorials/nlp/megatron_gpt_345m.nemo.
[NeMo I 2022-07-12 22:21:24 auto_tokenizer:171] 15 special tokens added, resize your model accordingly.


Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


In [42]:
# Training set to 10 epochs by default in a cell above
# Each epoch will take around 1min 15sec, but training time can vary
trainer.fit(model)

[NeMo W 2022-07-12 22:21:30 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:317: LightningDeprecationWarning: The `LightningModule.on_pretrain_routine_start` hook was deprecated in v1.6 and will be removed in v1.8. Please use `LightningModule.on_fit_start` instead.
      rank_zero_deprecation(
    
[NeMo W 2022-07-12 22:21:30 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/plugins/environments/torchelastic_environment.py:47: UserWarning: MASTER_ADDR environment variable is not defined. Set as localhost
      rank_zero_warn("MASTER_ADDR environment variable is not defined. Set as localhost")
    
[NeMo W 2022-07-12 22:21:30 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/plugins/environments/torchelastic_environment.py:55: UserWarning: MASTER_PORT environment variable is not defined. Set as 12910
      rank_zero_warn("MASTER_PORT environment variable is not defined. Set as

[NeMo I 2022-07-12 22:21:30 gpt_prompt_learning_dataset:63] Loading and tokenizing dataset ... 


1811it [00:00, 2125.20it/s]

[NeMo I 2022-07-12 22:21:31 gpt_prompt_learning_dataset:152] Skipped 0 sentences, sequence length too short or too long even after truncation



9960it [00:03, 2788.29it/s]

...

Epoch 9, global step 14710: 'val_loss' reached 0.77609 (best 0.77586), saving model to '/workspace/NeMo/tutorials/nlp/nemo_experiments/sentiment_intent_slot_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.776-step=14710.ckpt' as top 2


[NeMo I 2022-07-12 23:16:38 nlp_overrides:201] Removing checkpoint: /workspace/NeMo/tutorials/nlp/nemo_experiments/sentiment_intent_slot_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.778-step=11768.ckpt
[NeMo I 2022-07-12 23:16:42 nlp_overrides:201] Removing checkpoint: /workspace/NeMo/tutorials/nlp/nemo_experiments/sentiment_intent_slot_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.776-step=13239-last.ckpt
Epoch 9: 100%|██████████| 1566/1566 [05:21<00:00,  4.87it/s, loss=1.11, v_num=1-12, reduced_train_loss=1.030, global_step=14709.0, val_loss=0.776]

[NeMo W 2022-07-12 23:16:42 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2028: LightningDeprecationWarning: `Trainer.training_type_plugin` is deprecated in v1.6 and will be removed in v1.8. Use `Trainer.strategy` instead.
      rank_zero_deprecation(
    
Restoring states from the checkpoint path at /workspace/NeMo/tutorials/nlp/nemo_experiments/sentiment_intent_slot_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.776-step=13239.ckpt


Restored all states from the checkpoint file at /workspace/NeMo/tutorials/nlp/nemo_experiments/sentiment_intent_slot_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.776-step=13239.ckpt


[NeMo I 2022-07-12 23:16:44 megatron_gpt_prompt_learning_model:633] All p-tuned prompts where moved to the prompt table.
[NeMo I 2022-07-12 23:16:44 megatron_gpt_prompt_learning_model:646] The final model was saved to multitask_p_tuned_gpt.nemo


When training completes, p-tuned virtual tokens from the prompt encoder are automatically moved to a `prompt_table` where all prompt tuned and p-tuned soft prompts are stored. The LSTM `prompt_encoder` is then removed from the model. This allows us to preserve previously p-tuned soft prompts while still maintaining the ability to add new p-tuned or prompt-tuned soft prompts in the future. The `prompt_table` uses the `taskname` as a key to look up the correct virtual tokens for a specified task.

# Inference After First P-Tuning Session
One way to run inference after p-tuning or prompt-tuning your model is to call `model.generate()`. `model.generate()` takes in 

- `inputs` which can be either a list of dictionary objects or `.jsonl` files containing dictionary objects, 
- `length_params`
- `sampling_params`

as arguments. More information about the [text generation API can be found here](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/nlp/modules/common/transformer/text_generation.py).

If `length_params` and `sampling_params` are set to `None`, the model generates output with a greedy decoding strategy and generates up to `30` new tokens. Most predictive downstream tasks (not text generation tasks), use greedy sampling. To see other ways to run inference with your prompt learning model and more details on how to define various inference parameters, visit `examples/nlp/language_modeling/megatron_gpt_eval.py`.

Below are some randomly selected test examples from the sentiment classification and intent and slot classification test files. Notice that the `label` field is dropped from all test examples. The `MegatronPromptLearningDataset` called within `.generate()` automatically leaves fields in the prompt template empty when they are not provided in the data json. 

In [43]:
test_examples = [
    {"taskname": "intent_and_slot", "utterance": "tell me who will win the next presidential election"},
    {"taskname": "intent_and_slot", "utterance": "i would like to pickup a veggie sub with a cookie from subway"},
    {"taskname": "intent_and_slot", "utterance": "email happy new year to john"},
    {"taskname": "intent_and_slot", "utterance": "set the alarm to seven am for work"},
    {"taskname": "sentiment", "sentence": "The products have a low salt and fat content ."},
    {"taskname": "sentiment", "sentence": "The agreement is valid for four years ."},
    {"taskname": "sentiment", "sentence": "Diluted EPS rose to EUR3 .68 from EUR0 .50 ."},
    {"taskname": "sentiment", "sentence": "The company is well positioned in Brazil and Uruguay ."},
    {"taskname": "sentiment", "sentence": "Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier ."},
]

In [45]:
test_examples

[{'taskname': 'intent_and_slot',
  'utterance': 'tell me who will win the next presidential election'},
 {'taskname': 'intent_and_slot',
  'utterance': 'i would like to pickup a veggie sub with a cookie from subway'},
 {'taskname': 'intent_and_slot', 'utterance': 'email happy new year to john'},
 {'taskname': 'intent_and_slot',
  'utterance': 'set the alarm to seven am for work'},
 {'taskname': 'sentiment',
  'sentence': 'The products have a low salt and fat content .'},
 {'taskname': 'sentiment',
  'sentence': 'The agreement is valid for four years .'},
 {'taskname': 'sentiment',
  'sentence': 'Diluted EPS rose to EUR3 .68 from EUR0 .50 .'},
 {'taskname': 'sentiment',
  'sentence': 'The company is well positioned in Brazil and Uruguay .'},
 {'taskname': 'sentiment',
  'sentence': 'Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier .'}]

This allows us to prompt the p-tuned GPT model as follows:

```
"<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|> :\nplease will you check it."
"<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|> :\nset the alarm to seven am for work"
------------------
"<|VIRTUAL_PROMPT_0|> The products have a low salt and fat content . sentiment:"
"<|VIRTUAL_PROMPT_0|> The agreement is valid for four years . sentiment:"

```

With the correct virtual tokens inserted at each `<|VIRTUAL_PROMPT_#|>`

In [44]:
response = model.generate(inputs=test_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)

[NeMo I 2022-07-12 23:16:47 gpt_prompt_learning_dataset:63] Loading and tokenizing dataset ... 


100%|██████████| 9/9 [00:00<00:00, 908.36it/s]

[NeMo I 2022-07-12 23:16:47 gpt_prompt_learning_dataset:152] Skipped 0 sentences, sequence length too short or too long even after truncation



[NeMo W 2022-07-12 23:16:47 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/apex/transformer/pipeline_parallel/utils.py:81: UserWarning: This function is only for unittest
      warnings.warn("This function is only for unittest")
    


The prediction results of some sample queries with the trained model:
Predict intent and slot:
tell me who will win the next presidential election
Intent: general_negate
Slots: None
------------------------------
Predict intent and slot:
i would like to pickup a veggie sub with a cookie from subway
Intent: takeaway_order
Slots: food_type(veggie sub)
------------------------------
Predict intent and slot:
email happy new year to john
Intent: email_sendemail
Slots: person(john)
------------------------------
Predict intent and slot:
set the alarm to seven am for work
Intent: alarm_set
Slots: time(seven am)
------------------------------
The products have a low salt and fat content . sentiment: neutral
------------------------------
The agreement is valid for four years . sentiment: neutral
------------------------------
Diluted EPS rose to EUR3 .68 from EUR0 .50 . sentiment: positive
------------------------------
The company is well positioned in Brazil and Uruguay . sentiment: neutral


# Adding a New Task to a Previously Tuned Model
Now that we've p-tuned our GPT model on intent/slot classification and sentiment analysis, lets add SQuAD question answering using p-tuning! First we need to update the config for the new task. 

# Updating The Model Config
We need to update:

1. `name`
2. `model.restore_path`
3. `model.existing_tasks`
4. `model.new_tasks`
5. `model.virtual_prompt_style`
6. `model.data.train_ds`
7. `model.data.validation_ds`

Remember that we already set `task_templates` for SQuAD when we were defining the task template for the other two tasks. We would add it here if we had not already set it above.

In [46]:
# Change the experiment name
config.name = 'squad_p_tuning'

Here we tell the config that we want to **load the previously p-tuned model and add new tasks to it.**

In [47]:
# Change restore path from null to the p-tuned model we just finished training
config.model.restore_path = "multitask_p_tuned_gpt.nemo"

# Move the tasks you just p-tuned your model on to existing tasks, and add squad to the new task list
config.model.existing_tasks = ["sentiment", "intent_and_slot"]
config.model.new_tasks = ["squad"]

After the first round of p-tuning finished, the ``virtual_prompt_style`` got automatically set to ``inference`` at the end of training. This was done to make the prompt learning model ready as soon as training is complete. For the second round of p-tuning, we need to set ``virtual_prompt_style`` to ``p-tuning`` again.

In [48]:
# Reset virtual prompt style to "p-tuning" from "inference"
config.model.virtual_prompt_style = "p-tuning"

In [49]:
# Update the dataset list to squad train and val
# Using a subset of the training data for the sake of time
config.model.data.train_ds = [f"{SQUAD_DIR}/squad_short_train.jsonl"] 
config.model.data.validation_ds = [f"{SQUAD_DIR}/squad_val.jsonl"]

# Second P-Tuning Session

In [50]:
# Reset some model trainer and training params
config.trainer.max_epochs = 1
config.trainer.val_check_interval = 1000

# Limiting the number of validation batches for sake of time
config.trainer.limit_val_batches = 100

# Adjust learning rate for the task
config.model.optim.lr = 5e-4

# Reset the trainer
plugins = [NLPDDPPlugin(find_unused_parameters=False, no_ddp_communication_hook=True), TorchElasticEnvironment()]
trainer = pl.Trainer(plugins=plugins, **config.trainer)

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

[NeMo W 2022-07-12 23:42:19 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp.py:20: LightningDeprecationWarning: The `pl.plugins.training_type.ddp.DDPPlugin` is deprecated in v1.6 and will be removed in v1.8. Use `pl.strategies.ddp.DDPStrategy` instead.
      rank_zero_deprecation(
    
[NeMo W 2022-07-12 23:42:19 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:317: LightningDeprecationWarning: Passing <nemo.collections.nlp.parts.nlp_overrides.NLPDDPPlugin object at 0x7fe29cb612b0> `strategy` to the `plugins` flag in Trainer has been deprecated in v1.5 and will be removed in v1.7. Use `Trainer(strategy=<nemo.collections.nlp.parts.nlp_overrides.NLPDDPPlugin object at 0x7fe29cb612b0>)` instead.
      rank_zero_deprecation(
    
Using 16bit native Automatic Mixed Precision (AMP)
[NeMo W 2022-07-12 23:42:19 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytor

Trainer config - 

devices: 1
accelerator: gpu
num_nodes: 1
precision: 16
logger: false
enable_checkpointing: false
replace_sampler_ddp: false
max_epochs: 1
max_steps: null
log_every_n_steps: 10
val_check_interval: 1000
accumulate_grad_batches: 1
gradient_clip_val: 1.0
resume_from_checkpoint: null
benchmark: false
strategy: null
limit_val_batches: 100



In [51]:
from apex.transformer import parallel_state
from apex.transformer.pipeline_parallel.utils import _reconfigure_microbatch_calculator
from nemo.utils import AppState

app_state = AppState()

# Need to reconfigure micro batch calculator with apex for new p-tuning session
_reconfigure_microbatch_calculator(
    rank=app_state.global_rank,
    rampup_batch_size=None,
    global_batch_size=config.model.global_batch_size,
    micro_batch_size=config.model.micro_batch_size,
    data_parallel_size=parallel_state.get_data_parallel_world_size(),
)

[NeMo W 2022-07-12 23:42:20 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/apex/transformer/pipeline_parallel/utils.py:81: UserWarning: This function is only for unittest
      warnings.warn("This function is only for unittest")
    


In [52]:
# Reset experiment manager
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)
print(exp_dir)

[NeMo I 2022-07-12 23:42:22 exp_manager:287] Experiments will be logged at /workspace/NeMo/tutorials/nlp/nemo_experiments/squad_p_tuning/2022-07-12_22-21-12
[NeMo I 2022-07-12 23:42:22 exp_manager:661] TensorboardLogger has been set up


[NeMo W 2022-07-12 23:42:22 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2319: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.")
    
[NeMo W 2022-07-12 23:42:22 exp_manager:895] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.
[NeMo W 2022-07-12 23:42:22 exp_manager:698] Found save_best_model is True and save_nemo_on_train_end is False. Set save_nemo_on_train_end to True to automatically save the best model.


/workspace/NeMo/tutorials/nlp/nemo_experiments/squad_p_tuning/2022-07-12_22-21-12


In [53]:
# Restore previously tuned model with updated config
model = MegatronGPTPromptLearningModel.restore_from(config.model.restore_path, config.model, trainer=trainer)

[NeMo I 2022-07-12 23:42:23 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-07-12 23:42:23 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-07-12 23:42:23 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-07-12 23:42:23 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-07-12 23:42:23 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-07-12 23:42:23 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-07-12 23:42:23 megatron_init:231] All tensor model parallel group ranks: [[0]]
[NeMo I 2022-07-12 23:42:23 megatron_init:232] Rank 0 has tensor model parallel rank: 0
[NeMo I 2022-07-12 23:42:23 megatron_init:246] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2022-07-12 23:42:23 megatron_init:258] Rank 0 has embedding group: [0]
[NeMo I 2022-07-12 23:42:23 megatron_init:264] All pipeline model parallel group ranks: [[0]]
[NeMo I 2022-07-12 23:42:23 megatron_init:265]

[NeMo W 2022-07-12 23:42:25 modelPT:215] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-07-12 23:42:25 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-gpt-345m, custom vocab file: /tmp/tmp1jtkkcmu/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, and merges file: /tmp/tmp1jtkkcmu/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt
[NeMo I 2022-07-12 23:42:25 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: gpt2, vocab_file: /tmp/tmp1jtkkcmu/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, merges_files: /tmp/tmp1jtkkcmu/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt, special_tokens_dict: {}, and use_fast: False


Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2022-07-12 23:42:30 megatron_base_model:161] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
 > number of parameters on (tensor, pipeline) model parallel rank (0, 0): 354871296
[NeMo I 2022-07-12 23:42:30 nlp_overrides:367] Model MegatronGPTModel was successfully restored from /workspace/NeMo/tutorials/nlp/megatron_gpt_345m.nemo.
[NeMo I 2022-07-12 23:42:30 auto_tokenizer:171] 15 special tokens added, resize your model accordingly.


Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2022-07-12 23:42:30 save_restore_connector:243] Model MegatronGPTPromptLearningModel was successfully restored from /workspace/NeMo/tutorials/nlp/multitask_p_tuned_gpt.nemo.


In [54]:
# Prompt tune your model on squad
# This will take around 10 min per epoch, timing is variable
trainer.fit(model)

[NeMo W 2022-07-12 23:42:30 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:317: LightningDeprecationWarning: The `LightningModule.on_pretrain_routine_start` hook was deprecated in v1.6 and will be removed in v1.8. Please use `LightningModule.on_fit_start` instead.
      rank_zero_deprecation(
    


[NeMo I 2022-07-12 23:42:31 gpt_prompt_learning_dataset:63] Loading and tokenizing dataset ... 


14587it [00:17, 869.37it/s]

[NeMo I 2022-07-12 23:42:49 gpt_prompt_learning_dataset:230] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'squad'


20000it [00:24, 826.58it/s]

...

Epoch 0, global step 2000: 'val_loss' reached 0.83506 (best 0.83506), saving model to '/workspace/NeMo/tutorials/nlp/nemo_experiments/squad_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.835-step=2000.ckpt' as top 2


[NeMo I 2022-07-12 23:52:19 nlp_overrides:201] Removing checkpoint: /workspace/NeMo/tutorials/nlp/nemo_experiments/squad_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.898-step=1000-last.ckpt
Epoch 0: 100%|██████████| 2699/2699 [11:22<00:00,  3.95it/s, loss=0.73, v_num=1-12, reduced_train_loss=0.815, global_step=2498.0, val_loss=0.835] 

[NeMo W 2022-07-12 23:54:29 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2028: LightningDeprecationWarning: `Trainer.training_type_plugin` is deprecated in v1.6 and will be removed in v1.8. Use `Trainer.strategy` instead.
      rank_zero_deprecation(
    
Restoring states from the checkpoint path at /workspace/NeMo/tutorials/nlp/nemo_experiments/squad_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.835-step=2000.ckpt


Restored all states from the checkpoint file at /workspace/NeMo/tutorials/nlp/nemo_experiments/squad_p_tuning/2022-07-12_22-21-12/checkpoints/megatron_gpt_prompt_tune--val_loss=0.835-step=2000.ckpt


[NeMo I 2022-07-12 23:54:31 megatron_gpt_prompt_learning_model:633] All p-tuned prompts where moved to the prompt table.
[NeMo I 2022-07-12 23:54:31 megatron_gpt_prompt_learning_model:646] The final model was saved to multitask_p_tuned_gpt.nemo


# Inference After Second P-Tuning Session

Now we can run inference on all 3 tasks at once. The answers for the intent/slot and sentiment tasks should be identical to the ones from before p-tuning on squad. 

In [ ]:
#Test examples with squad examples added
test_examples = [
    {"taskname": "intent_and_slot", "utterance": "tell me who will win the next presidential election"},
    {"taskname": "intent_and_slot", "utterance": "i would like to pickup a veggie sub with a cookie from subway"},
    {"taskname": "intent_and_slot", "utterance": "email happy new year to john"},
    {"taskname": "intent_and_slot", "utterance": "set the alarm to seven am for work"},
    {"taskname": "sentiment", "sentence": "The products have a low salt and fat content ."},
    {"taskname": "sentiment", "sentence": "The agreement is valid for four years ."},
    {"taskname": "sentiment", "sentence": "Diluted EPS rose to EUR3 .68 from EUR0 .50 ."},
    {"taskname": "sentiment", "sentence": "The company is well positioned in Brazil and Uruguay ."},
    {"taskname": "sentiment", "sentence": "Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier ."},
    {"taskname": "squad", "context": "The build was released for download later in the day in standard 32-bit and 64-bit versions, plus a special 64-bit version which included SDKs and developer tools (Visual Studio Express and Expression Blend) for developing Metro-style apps. The Windows Store was announced during the presentation, but was not available in this build. According to Microsoft, there were about 535,000 downloads of the developer preview within the first 12 hours of its release. Originally set to expire on March 11, 2012, in February 2012 the Developer Preview's expiry date was changed to January 15, 2013.", "question": "When was the Developer preview initially intended to expire?"},
    {"taskname": "squad", "context": "The structures of most federal governments incorporate mechanisms to protect the rights of component states. One method, known as 'intrastate federalism', is to directly represent the governments of component states in federal political institutions. Where a federation has a bicameral legislature the upper house is often used to represent the component states while the lower house represents the people of the nation as a whole. A federal upper house may be based on a special scheme of apportionment, as is the case in the senates of the United States and Australia, where each state is represented by an equal number of senators irrespective of the size of its population.", "question": "What is a bicameral legislature?"},
    {"taskname": "squad", "context": "Imported mystery religions, which offered initiates salvation in the afterlife, were a matter of personal choice for an individual, practiced in addition to carrying on one's family rites and participating in public religion. The mysteries, however, involved exclusive oaths and secrecy, conditions that conservative Romans viewed with suspicion as characteristic of \"magic\", conspiratorial (coniuratio), or subversive activity. Sporadic and sometimes brutal attempts were made to suppress religionists who seemed to threaten traditional morality and unity, as with the senate's efforts to restrict the Bacchanals in 186 BC.", "question": "What was the practice of religion to the Romans?"}
]

In [ ]:
response = model.generate(inputs=test_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)

For squad, remember we only trained our model on ~29% of the training examples (20k instead of ~70k) and for only 1 epoch. Results will improve if the full training set is used and the model is tuned for more training steps.

This concludes our tutorial! For command line and script usage demos, [please see our docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/prompt_learning.html) 